
/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name
FROM Facilities
WHERE membercost > 0;


/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT(*) AS facilities_without_fee
FROM Facilities
WHERE membercost = 0;

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > 0 
  AND membercost < (monthlymaintenance * 0.2);


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
FROM Facilities
WHERE facid IN (1, 5);


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, 
       monthlymaintenance,
       CASE 
           WHEN monthlymaintenance > 100 THEN 'expensive'
           ELSE 'cheap'
       END AS cost_category
FROM Facilities;


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

**CONCAT broken in interface

SELECT DISTINCT 
    f.name AS court_name,
    CONCAT(m.firstname, ' ', m.surname) AS member_name
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
JOIN Members m ON b.memid = m.memid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY member_name;



/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT 
    f.name AS facility_name,
    CASE 
        WHEN b.memid = 0 THEN 'GUEST'
        ELSE CONCAT(m.firstname, ' ', m.surname)
    END AS member_name,
    CASE 
        WHEN b.memid = 0 THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots
    END AS cost
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
LEFT JOIN Members m ON b.memid = m.memid
WHERE DATE(b.starttime) = '2012-09-14'
  AND ((b.memid = 0 AND f.guestcost * b.slots > 30)
       OR (b.memid != 0 AND f.membercost * b.slots > 30))
ORDER BY cost DESC;

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT *
FROM (
    SELECT 
        f.name AS facility_name,
        CASE 
            WHEN b.memid = 0 THEN 'GUEST'
            ELSE CONCAT(m.firstname, ' ', m.surname)
        END AS member_name,
        CASE 
            WHEN b.memid = 0 THEN f.guestcost * b.slots
            ELSE f.membercost * b.slots
        END AS cost
    FROM Bookings b
    JOIN Facilities f ON b.facid = f.facid
    LEFT JOIN Members m ON b.memid = m.memid
    WHERE DATE(b.starttime) = '2012-09-14'
) AS bookings
WHERE cost > 30
ORDER BY cost DESC;

In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
#create generic functions to run the query

def execute_query(conn,query2):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()

    cur.execute(query2)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
        
def main2(query2):
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        execute_query(conn,query2)
        
        

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */


In [3]:
facilities_query = f'''SELECT f.name AS facility_name,SUM(CASE 
           WHEN b.memid = 0 THEN f.guestcost * b.slots
           ELSE f.membercost * b.slots
       END) AS total_revenue
FROM Facilities f
LEFT JOIN Bookings b ON f.facid = b.facid
GROUP BY f.facid, f.name
HAVING total_revenue < 1000
ORDER BY total_revenue;'''

In [4]:
main2(facilities_query)

2.6.0
2. Query all tasks
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [5]:
members_query = f'''SELECT m.surname, m.firstname, 
       r.surname AS recommender_surname, 
       r.firstname AS recommender_firstname
FROM Members m
LEFT JOIN Members r ON m.recommendedby = r.memid
WHERE m.memid != 0
ORDER BY m.surname, m.firstname;'''

In [6]:
main2(members_query)

2.6.0
2. Query all tasks
('Bader', 'Florence', 'Stibbons', 'Ponder')
('Baker', 'Anne', 'Stibbons', 'Ponder')
('Baker', 'Timothy', 'Farrell', 'Jemima')
('Boothe', 'Tim', 'Rownam', 'Tim')
('Butters', 'Gerald', 'Smith', 'Darren')
('Coplin', 'Joan', 'Baker', 'Timothy')
('Crumpet', 'Erica', 'Smith', 'Tracy')
('Dare', 'Nancy', 'Joplette', 'Janice')
('Farrell', 'David', None, None)
('Farrell', 'Jemima', None, None)
('Genting', 'Matthew', 'Butters', 'Gerald')
('Hunt', 'John', 'Purview', 'Millicent')
('Jones', 'David', 'Joplette', 'Janice')
('Jones', 'Douglas', 'Jones', 'David')
('Joplette', 'Janice', 'Smith', 'Darren')
('Mackenzie', 'Anna', 'Smith', 'Darren')
('Owen', 'Charles', 'Smith', 'Darren')
('Pinker', 'David', 'Farrell', 'Jemima')
('Purview', 'Millicent', 'Smith', 'Tracy')
('Rownam', 'Tim', None, None)
('Rumney', 'Henrietta', 'Genting', 'Matthew')
('Sarwin', 'Ramnaresh', 'Bader', 'Florence')
('Smith', 'Darren', None, None)
('Smith', 'Darren', None, None)
('Smith', 'Jack', 'Smith', 'Darr

/* Q12: Find the facilities with their usage by member, but not guests */

In [7]:
usage_mem_query = f'''SELECT f.name AS facility_name, 
       m.surname, 
       m.firstname, 
       SUM(b.slots) AS usage_slots
FROM Facilities f
JOIN Bookings b ON f.facid = b.facid
JOIN Members m ON b.memid = m.memid
WHERE b.memid != 0
GROUP BY f.facid, f.name, m.memid, m.surname, m.firstname
ORDER BY f.name, m.surname, m.firstname;'''

In [8]:
main2(usage_mem_query)

2.6.0
2. Query all tasks
('Badminton Court', 'Bader', 'Florence', 27)
('Badminton Court', 'Baker', 'Anne', 30)
('Badminton Court', 'Baker', 'Timothy', 21)
('Badminton Court', 'Boothe', 'Tim', 36)
('Badminton Court', 'Butters', 'Gerald', 63)
('Badminton Court', 'Crumpet', 'Erica', 6)
('Badminton Court', 'Dare', 'Nancy', 30)
('Badminton Court', 'Farrell', 'Jemima', 21)
('Badminton Court', 'Hunt', 'John', 6)
('Badminton Court', 'Jones', 'David', 24)
('Badminton Court', 'Jones', 'Douglas', 6)
('Badminton Court', 'Mackenzie', 'Anna', 96)
('Badminton Court', 'Owen', 'Charles', 18)
('Badminton Court', 'Pinker', 'David', 21)
('Badminton Court', 'Purview', 'Millicent', 6)
('Badminton Court', 'Rownam', 'Tim', 12)
('Badminton Court', 'Sarwin', 'Ramnaresh', 21)
('Badminton Court', 'Smith', 'Darren', 432)
('Badminton Court', 'Smith', 'Jack', 36)
('Badminton Court', 'Smith', 'Tracy', 102)
('Badminton Court', 'Stibbons', 'Ponder', 48)
('Badminton Court', 'Tracy', 'Burton', 6)
('Badminton Court', 'Tup

/* Q13: Find the facilities usage by month, but not guests */

In [9]:
usage_mon_query = f'''SELECT 
    f.name AS facility_name,
    EXTRACT(YEAR FROM b.starttime) AS year,
    EXTRACT(MONTH FROM b.starttime) AS month,
    SUM(b.slots) AS total_slots
FROM 
    Bookings b
JOIN 
    Facilities f ON b.facid = f.facid
JOIN 
    Members m ON b.memid = m.memid
WHERE 
    b.memid != 0  -- Exclude guests
GROUP BY 
    f.name,
    EXTRACT(YEAR FROM b.starttime),
    EXTRACT(MONTH FROM b.starttime)
ORDER BY 
    facility_name,
    year,
    month;'''
    

In [10]:
main2(usage_mem_query)

2.6.0
2. Query all tasks
('Badminton Court', 'Bader', 'Florence', 27)
('Badminton Court', 'Baker', 'Anne', 30)
('Badminton Court', 'Baker', 'Timothy', 21)
('Badminton Court', 'Boothe', 'Tim', 36)
('Badminton Court', 'Butters', 'Gerald', 63)
('Badminton Court', 'Crumpet', 'Erica', 6)
('Badminton Court', 'Dare', 'Nancy', 30)
('Badminton Court', 'Farrell', 'Jemima', 21)
('Badminton Court', 'Hunt', 'John', 6)
('Badminton Court', 'Jones', 'David', 24)
('Badminton Court', 'Jones', 'Douglas', 6)
('Badminton Court', 'Mackenzie', 'Anna', 96)
('Badminton Court', 'Owen', 'Charles', 18)
('Badminton Court', 'Pinker', 'David', 21)
('Badminton Court', 'Purview', 'Millicent', 6)
('Badminton Court', 'Rownam', 'Tim', 12)
('Badminton Court', 'Sarwin', 'Ramnaresh', 21)
('Badminton Court', 'Smith', 'Darren', 432)
('Badminton Court', 'Smith', 'Jack', 36)
('Badminton Court', 'Smith', 'Tracy', 102)
('Badminton Court', 'Stibbons', 'Ponder', 48)
('Badminton Court', 'Tracy', 'Burton', 6)
('Badminton Court', 'Tup